In [29]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Đường dẫn gốc 
BASE_DIR = Path.cwd()  
DATA_DIR = BASE_DIR / "dataset"

RAW_PATH   = DATA_DIR / "data.csv"         
CLEAN_PATH = DATA_DIR / "clean_data.csv"    

print("BASE_DIR :", BASE_DIR)
print("RAW_PATH :", RAW_PATH, "->", RAW_PATH.exists())
print("CLEAN_TO:", CLEAN_PATH)


BASE_DIR : d:\do-an-tot-nghiep
RAW_PATH : d:\do-an-tot-nghiep\dataset\data.csv -> True
CLEAN_TO: d:\do-an-tot-nghiep\dataset\clean_data.csv


In [30]:
df = pd.read_csv(RAW_PATH)

print("Số dòng:", len(df))
print("Các cột:", df.columns.tolist())
df.head()


Số dòng: 11534
Các cột: ['title', 'lead', 'body', 'url', 'subject', 'published_at', 'source', 'original_subject']


,title,lead,body,url,subject,published_at,source,original_subject
0,Tổng Bí thư: Chọn nhân sự Bộ Chính trị khóa 14...,Tổng Bí thư Tô Lâm nhấn mạnh nhân sự Bộ Chính ...,"Sáng 5/11, phát biểu khai mạc Hội nghị Trung ư...",https://vnexpress.net/tong-bi-thu-chon-nhan-su...,Chính trị,2025-11-05T09:54:00-07:00,vnexpress,Chính trị
1,Khai mạc Hội nghị Trung ương 14,Hội nghị lần thứ 14 Ban Chấp hành Trung ương k...,"Tại phiên khai mạc, Tổng Bí thư Tô Lâm chủ trì...",https://vnexpress.net/khai-mac-hoi-nghi-trung-...,Chính trị,2025-11-05T09:02:00-07:00,vnexpress,Chính trị
2,Tổng Bí thư: Không để người dân phải xin những...,Tổng Bí thư Tô Lâm nhấn mạnh Nhà nước pháp quy...,"Chiều 4/11, phát biểu trước Quốc hội về những ...",https://vnexpress.net/tong-bi-thu-khong-de-ngu...,Chính trị,2025-11-04T17:01:00-07:00,vnexpress,Chính trị
3,Phó chánh án Nguyễn Quốc Đoàn làm Phó tổng Tha...,"Ngày 4/11, Thủ tướng bổ nhiệm ông Nguyễn Quốc ...",Ông Đoàn đảm nhiệm cương vị mới thay ông Nguyễ...,https://vnexpress.net/pho-chanh-an-nguyen-quoc...,Chính trị,2025-11-04T16:42:00-07:00,vnexpress,Chính trị
4,Ông Nguyễn Văn Quảng làm Bí thư Đảng ủy Tòa án...,"Ông Nguyễn Văn Quảng, Phó tổng thường trực Tha...","Chiều 4/11, Ban Thường vụ Đảng ủy các cơ quan ...",https://vnexpress.net/ong-nguyen-van-quang-lam...,Chính trị,2025-11-04T15:58:00-07:00,vnexpress,Chính trị


In [31]:
text_cols = ["title", "lead", "body"]

for c in text_cols:
    if c in df.columns:
        missing = df[c].isna().mean() * 100
        print(f"Thiếu {c}: {missing:.2f}%")
    else:
        print(f"⚠️ Không thấy cột {c} trong data!")

# Chỉ giữ những dòng có body (và title nếu cần)
df = df[df["body"].astype(str).str.strip().str.len() > 0].copy()
df.reset_index(drop=True, inplace=True)

print("Sau khi bỏ body rỗng, còn:", len(df), "dòng")


Thiếu title: 0.00%
Thiếu lead: 0.18%
Thiếu body: 0.00%
Sau khi bỏ body rỗng, còn: 11534 dòng


Hàm chuẩn hoá text cơ bản


In [32]:
def normalize_text(x: str) -> str:
    if not isinstance(x, str):
        return ""
    
    # Chuẩn hoá xuống dòng → dấu chấm + khoảng trắng
    x = x.replace("\r\n", "\n")
    x = re.sub(r'\n+', '. ', x)

    # Chuẩn hoá khoảng trắng
    x = re.sub(r'\s+', ' ', x).strip()

    return x


Lọc rác



In [33]:
# Câu rác nguyên câu (caption, xem thêm, follow...)
PAT_SENT_DROP = [
    r"^\s*(xem thêm|đọc thêm|tham khảo|tin liên quan)\b.*$",
    r"^\s*(theo dõi|follow)\b.*\b(fanpage|kênh|zalo|tiktok|facebook|youtube)\b.*$",
    # Caption: câu bắt đầu bằng "Ảnh", "Hình ảnh", "Video", "Clip", "Infographic",...
    r"^\s*(ảnh minh hoạ|ảnh minh họa|ảnh|hình ảnh|video|clip|infographic|đồ hoạ|đồ họa)\b.*$",
]

# Rác inline trong ngoặc vuông/ngoặc tròn
PAT_INLINE_DROP = [
    r"\[(?:ảnh|hình ảnh|video|clip|infographic)[^\]]*\]",
    r"\((?:ảnh|hình ảnh|video|clip|infographic)[^)]*\)",
    r"【[^】]+】",
    r"〈[^〉]+〉",
]

# Ký hiệu bullet đầu câu
LEADING_MARKERS = re.compile(r"^\s*(?:>{2,}|-+|–+|—+|•+|\*+)\s*")

def strip_noise_segments(text: str) -> str:
   
    if not isinstance(text, str) or not text.strip():
        return ""

    # Tách câu đơn giản theo dấu câu
    sents = re.split(r"(?<=[\.!\?…])\s+", text.strip())
    cleaned = []

    for s in sents:
        s = LEADING_MARKERS.sub("", s.strip())
        if not s:
            continue

        # Bỏ cả câu nếu khớp pattern rác
        if any(re.search(p, s, flags=re.I) for p in PAT_SENT_DROP):
            continue

        # Xoá rác inline
        for p in PAT_INLINE_DROP:
            s = re.sub(p, " ", s, flags=re.I)

        # Chuẩn hoá khoảng trắng
        s = re.sub(r"\s+", " ", s).strip()
        if s:
            cleaned.append(s)

    out = ". ".join(cleaned)
    out = re.sub(r"\s*\.\s*\.\s*", ". ", out)   # gộp '... ...'
    out = re.sub(r"\s+", " ", out).strip(" .")
    return out


Áp dụng chuẩn hoá & lọc rác

In [34]:
print("ÁP DỤNG CHUẨN HÓA VÀ LỌC RÁC")

# Chuẩn hoá cơ bản
df["title"] = df["title"].map(normalize_text)
df["lead"]  = df["lead"].map(normalize_text)
df["body"]  = df["body"].map(normalize_text)

# Lọc rác trong lead & body
df["lead"] = df["lead"].map(strip_noise_segments)
df["body"] = df["body"].map(strip_noise_segments)

print("Sau clean text, còn:", len(df), "dòng (chưa lọc outlier).")

# In vài mẫu để nhìn
for i in range(3):
    print(f"\n--- MẪU {i} ---")
    print("TITLE:", df["title"].iloc[i])
    print("LEAD :", df["lead"].iloc[i])
    print("BODY :", df["body"].iloc[i][:300], "...")


ÁP DỤNG CHUẨN HÓA VÀ LỌC RÁC
Sau clean text, còn: 11534 dòng (chưa lọc outlier).

--- MẪU 0 ---
TITLE: Tổng Bí thư: Chọn nhân sự Bộ Chính trị khóa 14 phù hợp giai đoạn phát triển mới
LEAD : Tổng Bí thư Tô Lâm nhấn mạnh nhân sự Bộ Chính trị, Ban Bí thư khóa 14 phải phù hợp giai đoạn mới, có tầm nhìn chiến lược, năng lực chỉ huy, liêm chính đến mức biểu tượng, triển khai hiệu quả và sức bền vượt áp lực
BODY : Sáng 5/11, phát biểu khai mạc Hội nghị Trung ương 14, Tổng Bí thư Tô Lâm cho biết tại Hội nghị Trung ương 13, trên cơ sở đề nghị của Bộ Chính trị, Ban Chấp hành Trung ương khóa 13 đã biểu quyết thống nhất, tập trung cao, giới thiệu nhân sự Ban Chấp hành Trung ương khóa 14 - chưa bao gồm các Ủy viên  ...

--- MẪU 1 ---
TITLE: Khai mạc Hội nghị Trung ương 14
LEAD : Hội nghị lần thứ 14 Ban Chấp hành Trung ương khóa 13 khai mạc sáng 5/11, dự kiến kéo dài hai ngày để thảo luận về công tác Đại hội 14 và xây dựng Đảng, hệ thống chính trị
BODY : Tại phiên khai mạc, Tổng Bí thư Tô Lâm chủ tr

Thống kê độ dài body & xác định outlier

In [35]:
# Độ dài theo ký tự & từ
df["body_len_chars"] = df["body"].astype(str).str.len()
df["body_len_words"] = df["body"].astype(str).str.split().str.len()

print("Mô tả độ dài body (chars):")
print(df["body_len_chars"].describe(percentiles=[0.5, 0.9, 0.95, 0.99]))
print("\nMô tả độ dài body (words):")
print(df["body_len_words"].describe(percentiles=[0.5, 0.9, 0.95, 0.99]))

# Chọn ngưỡng outlier: top 1% dài nhất
q99_chars = df["body_len_chars"].quantile(0.99)
# Có thể đảm bảo min threshold cho chắc, ví dụ >= 8000 ký tự
min_threshold = 8000
max_body_len = int(max(q99_chars, min_threshold))

print(f"\nNgưỡng outlier (body quá dài) chọn theo 99th percentile hoặc >= {min_threshold}: {max_body_len} ký tự")
num_outliers = (df["body_len_chars"] > max_body_len).sum()
print(f"Số mẫu bị coi là outlier (body quá dài): {num_outliers}")


Mô tả độ dài body (chars):
count     11534.000000
mean       3001.850355
std        3893.261518
min         199.000000
50%        2597.000000
90%        4897.700000
95%        5883.050000
99%        8986.350000
max      341528.000000
Name: body_len_chars, dtype: float64

Mô tả độ dài body (words):
count    11534.000000
mean       651.341859
std        850.354375
min         45.000000
50%        562.000000
90%       1063.700000
95%       1270.000000
99%       1952.350000
max      74700.000000
Name: body_len_words, dtype: float64

Ngưỡng outlier (body quá dài) chọn theo 99th percentile hoặc >= 8000: 8986 ký tự
Số mẫu bị coi là outlier (body quá dài): 116


Lọc bỏ outlier & các mẫu body rỗng sau clean


In [36]:
# Lọc bỏ body quá dài (outlier)
before = len(df)
df = df[df["body_len_chars"] <= max_body_len].copy()
after = len(df)
print(f"Đã bỏ {before - after} mẫu outlier body quá dài. Còn lại: {after}")

# Bỏ các mẫu body/lead/title rỗng sau clean
df = df[
    (df["body"].astype(str).str.strip().str.len() > 0) &
    (df["title"].astype(str).str.strip().str.len() > 0)
].copy()

df.reset_index(drop=True, inplace=True)
print("Sau khi bỏ các mẫu body/title rỗng, còn:", len(df))


df = df[df["lead"].astype(str).str.strip().str.len() > 0].copy()
df.reset_index(drop=True, inplace=True)
print("Sau khi bỏ các mẫu lead rỗng, còn:", len(df))


Đã bỏ 116 mẫu outlier body quá dài. Còn lại: 11418
Sau khi bỏ các mẫu body/title rỗng, còn: 11418
Sau khi bỏ các mẫu lead rỗng, còn: 11383


Lưu clean_data.csv & kiểm tra

In [37]:
# Không cần giữ cột length trong file clean chính
df_clean = df.drop(columns=["body_len_chars", "body_len_words"], errors="ignore")

df_clean.to_csv(CLEAN_PATH, index=False, encoding="utf-8-sig")
print("Đã lưu clean_data.csv tới:", CLEAN_PATH)
print("Số dòng cuối cùng:", len(df_clean))

df_clean.head()


Đã lưu clean_data.csv tới: d:\do-an-tot-nghiep\dataset\clean_data.csv
Số dòng cuối cùng: 11383


,title,lead,body,url,subject,published_at,source,original_subject
0,Tổng Bí thư: Chọn nhân sự Bộ Chính trị khóa 14...,Tổng Bí thư Tô Lâm nhấn mạnh nhân sự Bộ Chính ...,"Sáng 5/11, phát biểu khai mạc Hội nghị Trung ư...",https://vnexpress.net/tong-bi-thu-chon-nhan-su...,Chính trị,2025-11-05T09:54:00-07:00,vnexpress,Chính trị
1,Khai mạc Hội nghị Trung ương 14,Hội nghị lần thứ 14 Ban Chấp hành Trung ương k...,"Tại phiên khai mạc, Tổng Bí thư Tô Lâm chủ trì...",https://vnexpress.net/khai-mac-hoi-nghi-trung-...,Chính trị,2025-11-05T09:02:00-07:00,vnexpress,Chính trị
2,Tổng Bí thư: Không để người dân phải xin những...,Tổng Bí thư Tô Lâm nhấn mạnh Nhà nước pháp quy...,"Chiều 4/11, phát biểu trước Quốc hội về những ...",https://vnexpress.net/tong-bi-thu-khong-de-ngu...,Chính trị,2025-11-04T17:01:00-07:00,vnexpress,Chính trị
3,Phó chánh án Nguyễn Quốc Đoàn làm Phó tổng Tha...,"Ngày 4/11, Thủ tướng bổ nhiệm ông Nguyễn Quốc ...",Ông Đoàn đảm nhiệm cương vị mới thay ông Nguyễ...,https://vnexpress.net/pho-chanh-an-nguyen-quoc...,Chính trị,2025-11-04T16:42:00-07:00,vnexpress,Chính trị
4,Ông Nguyễn Văn Quảng làm Bí thư Đảng ủy Tòa án...,"Ông Nguyễn Văn Quảng, Phó tổng thường trực Tha...","Chiều 4/11, Ban Thường vụ Đảng ủy các cơ quan ...",https://vnexpress.net/ong-nguyen-van-quang-lam...,Chính trị,2025-11-04T15:58:00-07:00,vnexpress,Chính trị


In [38]:
print("PHÂN PHỐI SUBJECT SAU LỌC")

subject_counts_after = df['subject'].value_counts().sort_index()
for subject, count in subject_counts_after.items():
    print(f"  {subject:30s}: {count:5d} mẫu ({count/len(df)*100:5.2f}%)")


print(f"TỔNG: {subject_counts_after.sum():5d} mẫu")


PHÂN PHỐI SUBJECT SAU LỌC
  Chính trị                     :  1020 mẫu ( 8.96%)
  Du lịch                       :  1011 mẫu ( 8.88%)
  Giáo dục                      :   979 mẫu ( 8.60%)
  Giải trí                      :  1049 mẫu ( 9.22%)
  Khoa học công nghệ            :  1008 mẫu ( 8.86%)
  Kinh doanh                    :  1094 mẫu ( 9.61%)
  Pháp luật                     :  1062 mẫu ( 9.33%)
  Sức khỏe                      :  1096 mẫu ( 9.63%)
  Thế giới                      :  1006 mẫu ( 8.84%)
  Thể thao                      :  1029 mẫu ( 9.04%)
  Đời sống                      :  1029 mẫu ( 9.04%)
TỔNG: 11383 mẫu
